In [1]:
# Loading libraries
import sys, os, json, re

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# Loading local library
rootPath = os.path.realpath(os.path.dirname(os.path.dirname(os.getcwd())))
libraryPath = os.path.join(rootPath, 'src', 'library')
sys.path.append(libraryPath)

import utility, k3s
import pandas

vocab = {}
vocabSize = len(vocab) 
sentences = []

In [3]:
def writeVocab(path, vocab):
    os.remove(path)
    df = pandas.Series(vocab).to_frame()
    #df = pandas.DataFrame(vocab, index=[0])
    #print(df)
    df.to_csv(path, mode='w', header = None)
    
def writeSentences(path, sentences):
    #np.savetxt('xgboost.txt', a.values, fmt='%d', delimiter="\t", header="X\tY\tZ\tValue") 
    df = pandas.DataFrame(sentences)
    #print(df)
    df.to_csv(path, mode='w', header = None)
    

In [4]:
dataPath = os.path.join(rootPath, 'data', 'bhot', 'raw')
charCode = {'a': '01',
           'b': '02',
           'c': '03',
           'd': '04',
           'e': '05',
           'f': '06',
           'g': '07',
           'h': '08',
           'i': '09',
           'j': '10',
           'k': '11',
           'l': '12',
           'm': '13',
           'n': '14',
           'o': '15',
           'p': '16',
           'q': '17',
           'r': '18',
           's': '19',
           't': '20',
           'u': '21',
           'v': '22',
           'w': '23',
           'x': '24',
           'y': '25',
           'z': '26',
           '-': '27'}

def encodeStrings(strs):
    processedStrs = {}
    for strItem in strs:
        strCode = ' '
        strItem = strItem.lower()
        
        for char in strItem:
            if charCode[char]:
                strCode += charCode[char]
            else:
                strCode += ' '
                
        processedStrs[word] = strCode
        
    return processedStrs

In [5]:
from nltk.stem.porter import PorterStemmer
'''
	1.	CC	Coordinating conjunction
	2.	CD	Cardinal number
	3.	DT	Determiner
	4.	EX	Existential there
	5.	FW	Foreign word
	6.	IN	Preposition or subordinating conjunction
	7.	JJ	Adjective
	8.	JJR	Adjective, comparative
	9.	JJS	Adjective, superlative
	10.	LS	List item marker
	11.	MD	Modal
	12.	NN	Noun, singular or mass
	13.	NNS	Noun, plural
	14.	NNP	Proper noun, singular
	15.	NNPS	Proper noun, plural
	16.	PDT	Predeterminer
	17.	POS	Possessive ending
	18.	PRP	Personal pronoun
	19.	PRP$	Possessive pronoun
	20.	RB	Adverb
	21.	RBR	Adverb, comparative
	22.	RBS	Adverb, superlative
	23.	RP	Particle
	24.	SYM	Symbol
	25.	TO	to
	26.	UH	Interjection
	27.	VB	Verb, base form
	28.	VBD	Verb, past tense
	29.	VBG	Verb, gerund or present participle
	30.	VBN	Verb, past participle
	31.	VBP	Verb, non-3rd person singular present
	32.	VBZ	Verb, 3rd person singular present
	33.	WDT	Wh-determiner
	34.	WP	Wh-pronoun
	35.	WP$	Possessive wh-pronoun
	36.	WRB	Wh-adverb
'''
    
def getFilteredWords(text):
    stemmer = PorterStemmer()
    processedWords = []
    #allowedPOSTypes = ['NN', 'NNP', 'NNS', 'NNPS', 'JJ', 'JJR', 'JJS' 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    allowedPOSTypes = ['NN', 'NNP', 'NNS', 'NNPS']
    
    lc = k3s.LC(text)
    words = lc.getWords(text, True)
    currentSentence = []
    for word in words:
        (word, type) = word
        word = re.sub('[^a-zA-Z]+', '', word)
        if type in ['.', '?', '!']:
            if len(currentSentence) > 1:
                # If more than one word than add as sentence
                sentences.append(' '.join(currentSentence))
            currentSentence = []
        if len(word) < 2:
            continue
        if type in allowedPOSTypes:
            #print(type + ' ' + word)
            word = word.lower()
            word = stemmer.stem(word)
            processedWords.append(word)
            if word not in currentSentence:
                currentSentence.append(word)
        
    return processedWords
    

In [6]:
def buildVocabulary(text):
    global vocabSize
    for word in text:
        if word in vocab:
            continue
        vocab[word] = vocabSize
        vocabSize += 1


In [7]:


def processTextFiles(directoryPath):
    
    print('Processing directory: ', directoryPath)
    for root, dirs, files in os.walk(dataPath):
        for file in files:
            if file.endswith(".txt"):
                #print('---------------------------------------')
                file = utility.File(os.path.join(root, file))
                text = file.read()
                textWords = getFilteredWords(text)
                buildVocabulary(textWords)
                #print(len(textWords))
                #print(vocabSize)
                #sys.exit
                #lc = k3s.LC(text)
                #lc.process()
                #contributers = lc.getContributers()
                #result = {}
                #result['clean_text'] = lc.getCleanText()
                #result['most_important_featured_words'] = ",".join(contributers[0])
                #result['other_featured_words'] = ",".join(contributers[1])
                #print(encodeWords(contributers[0]))
                #print(lc.getWords(text, True))
                #print(result)
                                

In [8]:

bhotPath = os.path.join(rootPath, 'data', 'bhot', 'vocab.csv')
bhotTrainPath = os.path.join(rootPath, 'data', 'bhot', 'train.csv')

processTextFiles(dataPath)
writeVocab(bhotPath, vocab)
writeSentences(bhotTrainPath, sentences)
print('Files generated')
# Reverse dictionary
#print(dict(map(reversed, vocab.items())))

Processing directory:  /notebooks/K3S/data/bhot/raw
Files generated


Wordnet:  https://wordnet.princeton.edu/